In [1]:
import pandas as pd
import os

In [2]:
glasses_df = pd.read_csv('2024-binary-classification-glasses-no-glasses.csv')
decimal_df = pd.read_csv('2024-decimal-classification-mics-mnist.csv')
cartoon_df = pd.read_csv('2024-cartoon-to-image.csv')
upscale_df = pd.read_csv('2024-upscale.csv')
function1_df = pd.read_csv('2024-fitting-a-1d-function-with-deep-learning.csv')
function3_df = pd.read_csv('2024-fitting-a-3d-3d-function-with-deep-learning.csv')

In [3]:
from Levenshtein import distance
from unidecode import unidecode


In [4]:
distance("lewenstein", "levenshtein"), unidecode("é")

(2, 'e')

In [5]:
with open('names.txt', encoding='UTF8') as f:
    names = f.read()
names = names.split("\n")
names = [unidecode(name).lower() for name in names]

In [6]:
# remove row with "submission" in the team name
glasses_df = glasses_df[~glasses_df['TeamName'].str.contains("submission")]
decimal_df = decimal_df[~decimal_df['TeamName'].str.contains("submission")]
cartoon_df = cartoon_df[~cartoon_df['TeamName'].str.contains("submission")]
upscale_df = upscale_df[~upscale_df['TeamName'].str.contains("submission")]
function1_df = function1_df[~function1_df['TeamName'].str.contains("submission")]
function3_df = function3_df[~function3_df['TeamName'].str.contains("submission")]

In [7]:
def find_closest(name, names):
    distances = [min(distance(unidecode(name).replace('_', ' '), name_),\
                     distance(unidecode(name).replace('_', ' '), ' '.join(name_.split()[::-1])))\
                          for name_ in names]
    return names[distances.index(min(distances))]

def find_closest_df(df, names):
    df['name'] = df['TeamName'].apply(lambda x: find_closest(x, names))
    return df

glasses_df = find_closest_df(glasses_df, names)
decimal_df = find_closest_df(decimal_df, names)
cartoon_df = find_closest_df(cartoon_df, names)
upscale_df = find_closest_df(upscale_df, names)
function1_df = find_closest_df(function1_df, names)
function3_df = find_closest_df(function3_df, names)


In [8]:
# calculate the percentil rank
glasses_df['Percentil'] = 1-((glasses_df['Rank']-1)/glasses_df.shape[0])
decimal_df['Percentil'] = 1-((decimal_df['Rank']-1)/decimal_df.shape[0])
cartoon_df['Percentil'] = 1-((cartoon_df['Rank']-1)/cartoon_df.shape[0])
upscale_df['Percentil'] = 1-((upscale_df['Rank']-1)/upscale_df.shape[0])
function1_df['Percentil'] = 1-((function1_df['Rank']-1)/function1_df.shape[0])
function3_df['Percentil'] = 1-((function3_df['Rank']-1)/function3_df.shape[0])

In [9]:
def get_line(name, df):
    return df[df['name'] == name]

get_line(names[0], glasses_df)

,Rank,TeamId,TeamName,LastSubmissionDate,Score,SubmissionCount,TeamMemberUserNames,name,Percentil
2,3,11798408,AbdallahLamane,2024-03-18 21:28:34,0.99466,2,abdallahlamane,abdallah lamane,0.875


In [10]:
def get_mark1(name):
    global function1_df, function3_df
    try:
        function1_score = float(get_line(name, function1_df).Score.iloc[0])
        function1_rank = float(get_line(name, function1_df).Rank.iloc[0])
        if function1_score < 1:
            function1_mark = 0.5
        if function1_score < 0.1:
            function1_mark = 0.7
        if function1_score < 0.01:
            function1_mark = 0.9
        if function1_rank <= 10:
            function1_mark = 1
    except:
        function1_mark = 0
    try:
        function3_score = float(get_line(name, function3_df).Score.iloc[0])
        function3_mark = 0
        if function3_score < 10_000_000 :
            function3_mark = 0.5
        if function3_score < 5_000_000 :
            function3_mark = 0.6
        if function3_score < 1_000_000 :
            function3_mark = 0.7
        if function3_score < 500_000 :
            function3_mark = 0.8
        if function3_score < 100_000 :
            function3_mark = 0.9
        if function3_score < 50_000:
            function3_mark = 1
    except:
        function3_mark = 0
    return function1_mark, function3_mark

get_mark1(names[0])

(0.9, 0)

In [11]:
def get_mark2(name):
    global glasses_df, decimal_df
    try:
        glass_mark = max(float(get_line(name, glasses_df).Score.iloc[0]), \
                         float(get_line(name, glasses_df).Percentil.iloc[0]))
    except:
        glass_mark = 0

    try:
        decimal_mark = max(float(get_line(name, decimal_df).Score.iloc[0]), \
                           float(get_line(name, decimal_df).Percentil.iloc[0]))
    except:
        decimal_mark = 0
    return glass_mark, decimal_mark

get_mark1(names[0])

(0.9, 0)

In [12]:
def get_mark3(name):
    global upscale_df, cartoon_df
    try:
        upscale_score = float(get_line(name, upscale_df).Score.iloc[0])
        upscale_rank = float(get_line(name, upscale_df).Rank.iloc[0])
        upscale_mark = 0
        if upscale_score < 15.80364:
            upscale_mark = 1
    except:
        upscale_mark = 0
    try:
        cartoon_score = float(get_line(name, cartoon_df).Score.iloc[0])
        upscale_rank = float(get_line(name, upscale_df).Rank.iloc[0])
        cartoon_mark = 0
        if cartoon_score < 44.85942 :
            cartoon_mark = 0.5
        if upscale_rank <= 20 :
            cartoon_mark = 0.6
        if upscale_rank <= 15 :
            cartoon_mark = 0.7
        if upscale_rank <= 10 :
            cartoon_mark = 0.8
        if upscale_rank <= 5 :
            cartoon_mark = 1
    except:
        cartoon_mark = 0
    return upscale_mark, cartoon_mark

get_mark3(names[0])

(1, 0)

In [13]:
k1 = []
k2 = []
k3 = []
for name in names:
    k1.append(max(get_mark1(name)))
    k2.append(max(get_mark2(name)))
    k3.append(max(get_mark3(name)))

# make df from k1, k2, k3
# df = pd.DataFrame({'name': names, 'k1': k1, 'k2': k2, 'k3': k3})
df = pd.DataFrame({'k1': k1, 'k2': k2, 'k3': k3})
df.to_csv('final-grades.csv', index=False)